In [30]:
class Dealer():
    def __init__(self, deck):
        # initialize empty hand
        self.hand = []
        self.ace = False
        self.above_21 = False
        # get 2 card by .hit_card() method of deck
        for n in range(0,2):
            self.card = deck.hit_card()
            self.hand.append(self.card)
            if self.card.rank == 'Ace':
                self.ace = True
        self. value = sum([card.value for card in self.hand])
        
    # moves to do: hit or stand
    def hit(self, deck):
        self.card = deck.hit_card()
        print('Card drawn: ')
        print(self.card.suite, self.card.rank, self.card.value)
        self.hand.append(self.card)
        self.value = sum([card.value for card in self.hand])
        if self.value > 21 and self.ace == True:
            self.value -=10
        elif self.value >21 and self.ace == False:
            self.above_21 = True
        return self.value
        
    
    # one card of dealers hand is visible
    def show_dealer(self):
        card = self.hand[0]
        print('dealers got: ', card.suite, card.rank, card.value)

    # For development - show hand and value etc
    def show_hand(self):
        for card in self.hand:
            print(card.suite, card.rank, card.value)
        print('hand value: ', self.value)
        print('have ace ?: ', self.ace)

In [31]:
class Player(Dealer):
        def __init__(self, deck, money, name):
                super().__init__(deck)
        # Player inherit from Dealer but need additional attribute money
                self.money = money
                self.name = name
                self.queue = 'initialization'
                self.next = None
        # loose or win money - depends on conditions checking
        def set_queue(self, value):
               if value in ["Active","Waiting","Standing"]:
                      self.queue = value
               else:
                     print('unknown atribute')
        def set_next(self, player):
               self.next = player
        def win_money(self, ammount):
               self.money += ammount
        # For development - check money
        def check_money(self):
            print('You ve got, ', self.money)


In [32]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

In [33]:
class Card():
    def __init__(self, suite, rank):
        self.suite= suite
        self.rank= rank
        self.value= values[rank]

In [34]:
import random
class Deck():
    def __init__(self):
        self.set_of_all=[]
        for suite in suits:
            for rank in ranks:
                self.set_of_all.append(Card(suite, rank))
        #always shuffle new set
        self.shuffle()
    def shuffle(self):
        print('deck shuffling')
        print(len(self.set_of_all), ' cards in deck')
        random.shuffle(self.set_of_all)
    def hit_card(self):
        return self.set_of_all.pop()
    def __str__(self):
        return str(len(self.set_of_all))

In [35]:
class Game_queue():
    def __init__(self):
        self.list_of_players=[]
        self.is_active = False
    def add_player(self,player):
      player.set_queue("Waiting")
      self.list_of_players.append(player)
      if self.is_active == False:
                    player.set_queue("Active")
                    self.is_active = True
    # set queue
    # if new player is added -> he becomes next for previous player
    # AND player 0 must become next for last in list (so it goes in circle)
      if len(self.list_of_players) <= 1:
          player.set_next(player)
      elif len(self.list_of_players) > 1:
          self.list_of_players[-2].set_next(player)
          self.list_of_players[-1].set_next(self.list_of_players[0])
    def check_all(self):
          for player in self.list_of_players:
                print(player.queue)
    def check_queue(self):
          for player in self.list_of_players:
                print(player.name, player.queue, " next - " , player.next.name)
# look for active player, then push "Active status to next one"
    def move_queue(self):
          for player in self.list_of_players:
                if player.queue == 'Active':
                  print('move queue', player.name, player.queue, " next - " , player.next.name)
                  player.set_queue("Waiting")
                  player.next.set_queue("Active")
                  break
    def player_stand(self):
              for player in self.list_of_players:
                  if player.queue == 'Active':
                    player.set_queue("Standing")
                    queue_status = set([player.queue for player in self.list_of_players])
                    queue_status = list(queue_status)
                    if (queue_status == ["Standing"]) == True:
                      break
                    else:
                      player.next.set_queue("Active")
                      break
    def whos_turn(self):
          for player in self.list_of_players:
              if player.queue == "Active":
                   return player
          print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
          print('no active player')
          return False

In [36]:
q = Game_queue()
deck = Deck()
player1 = Player(deck, 100, 'Player1')
player2 = Player(deck, 100, 'Player2')
player3 = Player(deck, 100, 'Player3')
player4 = Player(deck, 100, 'Player4')

q.add_player(player1)
q.add_player(player2)
q.add_player(player3)
q.add_player(player4)

deck shuffling
52  cards in deck


In [37]:
q.player_stand()
q.check_queue()

Player1 Standing  next -  Player2
Player2 Active  next -  Player3
Player3 Waiting  next -  Player4
Player4 Waiting  next -  Player1


In [38]:
player1.set_queue('Active')

Game engine

In [42]:
import time
from IPython.display import clear_output
class Game_engine():

    def __init__(self, dealer, Game_queue, deck):
        print('Game starts')
        self.dealer = dealer
        self.deck = deck
        self.bet = []
        self.Game_queue = Game_queue
        self.whos_turn = {'Player':True, 'Dealer':False}
        self.input=''


    def game_turn(self):
        self.player = self.Game_queue.whos_turn()

        if self.player != False:
            self.input = input(f'{self.player.name} Turn: h or s or hand')
            print('you choose', self.input)
        # HIT!
            if self.input == 'h':
                self.player.hit(self.deck)
                print(self.player.name, ' value: ', self.player.value)
                if self.player.value >21:
                    print(self.player,' loose')
                    self.Game_queue.player_stand()
        # STAND!
            elif self.input == 's':
                print(self.player.name ,'value: ', self.player.value)
                #clear_output(wait=True)
                self.Game_queue.player_stand()
        # CHECK HAND
            elif self.input == 'hand':
                self.player.show_hand()
                self.dealer.show_dealer()
            self.game_turn()
        
        elif self.input == 'e':
                self.exit()    
            
# DEALERS TURN if below 17 - hit, over 17 stand
        if self.player == False:
            print(self.dealer.show_hand())
            if self.dealer.value < 17:
                self.dealer.hit(self.deck)
                print('dealr hits',  self.dealer.value)
                self.game_turn()
            elif self.dealer.value >= 17:
                print('dealer stand',  self.dealer.value)
                self.check_bet()

    def check_bet(self):
# WIN/LOOSE CONDITION
       print('checking who won')
       print('so far i`m finishing :)')
       self.exit()

    # scenario if player loose
    def player_loose(self):
        print('player lost')
        print('reinitialising player dealer and bet')
        self.bet = []
        self.player.__init__(self.deck, self.player.money)
        self.dealer.__init__(self.deck)
        if self.whos_turn['Player'] == False:
            self.switch_turn()

    def player_win(self):
        print('player wins')
        self.player.win_money(sum(self.bet))
        print('reinitialising player dealer and bet')
        self.bet = []
        self.player.__init__(self.deck, self.player.money)
        self.dealer.__init__(self.deck)
        if self.whos_turn['Player'] == False:
            self.switch_turn()

    def exit(self):
        print('exit')

In [43]:
q = Game_queue()
deck = Deck()
player1 = Player(deck, 100, 'Player1')
player2 = Player(deck, 100, 'Player2')
player3 = Player(deck, 100, 'Player3')
player4 = Player(deck, 100, 'Player4')

q.add_player(player1)
q.add_player(player2)
q.add_player(player3)
q.add_player(player4)

deck = Deck()
dealer = Dealer(deck)
game = Game_engine( Game_queue=q, dealer=dealer, deck=deck)


deck shuffling
52  cards in deck
deck shuffling
52  cards in deck
Game starts


In [44]:
game.game_turn()

you choose s
Player1 value:  10
you choose s
Player2 value:  18
you choose s
Player3 value:  21
you choose s
Player4 value:  19
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
no active player
Diamonds Three 3
Hearts King 10
hand value:  13
have ace ?:  False
None
Card drawn: 
Clubs Four 4
dealr hits 17
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
no active player
Diamonds Three 3
Hearts King 10
Clubs Four 4
hand value:  17
have ace ?:  False
None
dealer stand 17
checking who won
so far i`m finishing :)
exit
Diamonds Three 3
Hearts King 10
Clubs Four 4
hand value:  17
have ace ?:  False
None
dealer stand 17
checking who won
so far i`m finishing :)
exit
Diamonds Three 3
Hearts King 10
Clubs Four 4
hand value:  17
have ace ?:  False
None
dealer stand 17
checking who won
so far i`m finishing :)
exit
Diamonds Three 3
Hearts King 10
Clubs Four 4
hand value:  17
have ace ?:  False
None
dealer stand 17
checking who won
so far i`m finishing :)
exit
Diamonds Three 3
Hearts King 10
Clubs Four 4
hand value:  17
have ace 